In [7]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from pathlib import Path
from tqdm import tqdm
import json
from bson import ObjectId
from pprint import pprint
from datetime import datetime


sys.path.append(str(Path("..").resolve()))
from src import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Update Queries
ℹ️ This notebook require the execution of [`1.3-Consistency_Recover`](../1-migration/1.3-Consistency_Recover.ipynb). Or you can just load the snapshop as in the following cell.

## Load post-preprocessing data
The following cell reset the `omero_museum` database loading the `4_consistent` snapshot.

In [ ]:
%%capture
MongoClient("mongodb://localhost:27017/").drop_database("omero_museum")
!mongorestore --host localhost:27017 --drop --db omero_museum  "../backup/5_optimized/omero_museum"

In [4]:
connector = MongoDBConnector("omero_museum")
db = connector.db

The collections of the omero_museum db are:
----------------------------------------
[activities]: _id capacity duration enrolled room startDate ticketIds workshopTitle
[artworks]: _id authorIds comments_star_1 comments_star_2 comments_star_3 comments_star_4 comments_star_5 date description donationState donatorId isOriginal locationName materials period sellerId size tecniques trade type
[authors]: _id birth_date gender home_town name surname
[departments]: _id floor free_spots room
[employees]: _id birth_date curriculum date_start email gender hometown name phone_number surname
[limited_events]: _id artist artworkTitles authorIds capacity description endDate roomName startDate theme ticketIds type
[messages]: _id content customerId deliveryStatus length sentDate type uri
[rooms]: _id floor
[suppliers]: _id email iban isMuseum isState name phoneNum surveys
[visitors]: _id birthDate chat comments donationIds email gender impairment isCustomer land name phoneNum saleIds surname surveys 

### UP1: _Update the description of a material_

In [9]:
db.artworks.update_many(
    {"materials": "Marble"},
    [
        {"$set": {
            "materials": {
                "$map": {
                    "input": "$materials",
                    "as": "m",
                    "in": {
                        "$cond": [
                            {"$eq": ["$$m", "Marble"]},
                            "White Marble",
                            "$$m"
                        ]
                    }
                }
            }
        }}
    ]
)


UpdateResult({'n': 5, 'nModified': 5, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### UP2: _Check-in a visitor's ticket_

In [ ]:
ticket_date = datetime(2025, 9, 2, 13, 7, 37)

db.visitors.update_one(
    {"_id": 0},
    [
        {
            "$set": {
                "tickets": {
                    "$map": {
                        "input": "$tickets",
                        "as": "t",
                        "in": {
                            "$cond": [
                                {"$eq": ["$$t.date", ticket_date]},
                                {**{"check_in": True}, **"$$t"},
                                "$$t"
                            ]
                        }
                    }
                }
            }
        }
    ]
)


### UP3: _Update trade information of an artwork_

In [15]:
db.artworks.update_one(
    {"_id": "La Notte Stellata"}, 
    {"$set": {
        "trade.win_price": 5000,
        "trade.winner_id": 1,
    }}
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### UP4: _Update information of an artwork_

In [12]:
db.artworks.update_one(
    {"_id": "La Notte Stellata"}, 
    {"$set": {
        "isOriginal": False,
        "tecniques": ["painting"],
        "materials": ["Glass", "Bronze"],
    }}
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### UP5: _Update curriculum of an employee_

In [8]:
employee_id = ObjectId("68b572f8845fcdbabc1c8930")
new_curriculum = "https://linkedin.com/in/matilda-scerbo"

db.employees.update_one(
    {"_id": employee_id},
    [
        {
            "$set": {
                "curriculum": new_curriculum
            }
        }
    ]
)


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### UP6: _Update employee in a department_

### UP7: _Update the shift of an given employee_

### UP8: _Change the date of a limited event_

In [10]:
db.limited_events.update_one(
    {"_id": "The Future of Mosaic"}, 
    {"$set": {
        "startDate": datetime(2025, 10, 2, 10, 0),
        "endDate": datetime(2025, 10, 2, 18, 0),
    }}
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### UP9: _Update a message in a given chat_

In [ ]:
target_customer_id = 10
target_sent_date = datetime(2025, 9, 1, 0, 44, 49)
new_content = "A new message content"

db.visitors.update_one(
    {"_id": 0},
    [
        {
            "$set": {
                "chat.new_messages": {
                    "$map": {
                        "input": "$chat.new_messages",
                        "as": "msg",
                        "in": {
                            "$cond": [
                                {
                                    "$and": [
                                        {"$eq": ["$$msg.customerId", target_customer_id]},
                                        {"$eq": ["$$msg.sentDate", target_sent_date]}
                                    ]
                                },
                                {
                                    "$mergeObjects": [
                                        "$$msg",
                                        {
                                            "content": new_content,
                                            "sentDate": datetime.now()
                                        }
                                    ]
                                },
                                "$$msg"
                            ]
                        }
                    }
                }
            }
        }
    ]
)


UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### UP10: _Update prices of a workshop_

In [4]:
db.workshops.update_one(
    {"_id": "Pittura su tela"}, 
    {"$set": {
        "price_person": 30.00,
        "price_class": 90.00
    }}
)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)